# Acrobot

## A2C Agent 

In [22]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "Acrobot-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "Acrobot-v1":
    gamma = 0.95
    lr = 7e-4

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -500.0
Episode 9: -500.0
Episode 10: -500.0
Episode 11: -500.0
Episode 12: -500.0
Episode 13: -500.0
Episode 14: -500.0
Episode 15: -500.0
Episode 16: -500.0
Episode 17: -500.0
Episode 18: -500.0
Episode 19: -500.0
Episode 20: -500.0
Episode 21: -500.0
Episode 22: -500.0
Episode 23: -500.0
Episode 24: -500.0
Episode 25: -500.0
Episode 26: -500.0
Episode 27: -500.0
Episode 28: -500.0
Episode 29: -500.0
Episode 30: -500.0
Episode 31: -500.0
Episode 32: -500.0
Episode 33: -500.0
Episode 34: -500.0
Episode 35: -500.0
Episode 36: -500.0
Episode 37: -500.0
Episode 38: -500.0
Episode 39: -500.0
Episode 40: -500.0
Episode 41: -500.0
Episode 42: -500.0
Episode 43: -500.0
Episode 44: -500.0
Episode 45: -500.0
Episode 46: -500.0
Episode 47: -500.0
Episode 48: -500.0
Episode 49: -500.0
Episode 50: -500.0
Episode 51: -500.0
Episode 52: -500.0
Epi

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [17]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = "Acrobot-v1"
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent
if kl_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    beta = 0.8

agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        avg_episode_reward = episode_reward/env.action_space.n
    
    # add randomness for better exploration
    beta += np.random.random()*0.1
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward  <= -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    if episode % 10 == 0:
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        state_adv[2] += (np.random.random()-0.5)*0.5

    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -391.6666666666667
Episode 3: -281.0
Episode 4: -304.3333333333333
Episode 5: -228.33333333333334
Episode 6: -230.0
Episode 7: -153.66666666666666
Episode 8: -188.33333333333334
Episode 9: -135.33333333333334
Episode 10: -158.66666666666666
Episode 11: -155.33333333333334
Episode 12: -137.33333333333334
Episode 13: -145.0
Episode 14: -144.33333333333334
Episode 15: -138.0
Episode 16: -130.33333333333334
Episode 17: -115.0
Episode 18: -119.0
Episode 19: -245.33333333333334
Episode 20: -118.66666666666667
Episode 21: -107.33333333333333
Episode 22: -112.66666666666667
Episode 23: -85.33333333333333
Episode 24: -164.33333333333334
Episode 25: -98.66666666666667
Episode 26: -112.0
Episode 27: -102.66666666666667
Episode 28: -112.33333333333333
Episode 29: -105.0
Episode 30: -127.66666666666667
Episode 31: -97.66666666666667
Episode 32: -93.0
Episode 33: -120.66666666666667
Episode 34: -102.0
Episode 35: -151.66666666666666
Episode 36: -99.3333

Episode 298: -101.0
Episode 299: -110.0


In [20]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [18]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "Acrobot-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "Acrobot-v1":
    gamma = 0.95
    lr = 5e-3
    
agent = DRTRPOAgent(env, gamma, lr)

############################### MC Updates  (Full Episode) ###############################

# Define training parameters
max_episodes = 300
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        avg_episode_reward = episode_reward/env.action_space.n
    
    # add randomness for better exploration 
    if(state_adv[0] == state_adv[1]) and (state_adv[1] == state_adv[2]) and avg_episode_reward < -490:
        state_adv[0] += (np.random.random()-0.5)*2
        state_adv[1] += (np.random.random()-0.5)*2
        state_adv[2] += (np.random.random()-0.5)*2
        
    total_adv_diff += max(abs(state_adv[1] - state_adv[0]), abs(state_adv[2] - state_adv[0]), abs(state_adv[2] - state_adv[1]))
    beta = total_adv_diff/episode
    beta += 0.1
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta)

    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: -500.0
Episode 1: -500.0
Episode 2: -500.0
Episode 3: -500.0
Episode 4: -500.0
Episode 5: -500.0
Episode 6: -500.0
Episode 7: -500.0
Episode 8: -474.3333333333333
Episode 9: -500.0
Episode 10: -442.3333333333333
Episode 11: -500.0
Episode 12: -490.0
Episode 13: -454.0
Episode 14: -424.0
Episode 15: -475.3333333333333
Episode 16: -428.3333333333333
Episode 17: -474.3333333333333
Episode 18: -494.0
Episode 19: -463.6666666666667
Episode 20: -466.6666666666667
Episode 21: -458.6666666666667
Episode 22: -466.6666666666667
Episode 23: -407.3333333333333
Episode 24: -466.6666666666667
Episode 25: -484.6666666666667
Episode 26: -460.6666666666667
Episode 27: -407.3333333333333
Episode 28: -440.0
Episode 29: -401.3333333333333
Episode 30: -452.3333333333333
Episode 31: -488.6666666666667
Episode 32: -436.0
Episode 33: -417.3333333333333
Episode 34: -461.6666666666667
Episode 35: -462.0
Episode 36: -460.0
Episode 37: -422.6666666666667
Episode 38: -417.0
Episode 39: -493.666666666666

Episode 292: -148.33333333333334
Episode 293: -119.0
Episode 294: -166.66666666666666
Episode 295: -137.33333333333334
Episode 296: -137.0
Episode 297: -172.33333333333334
Episode 298: -142.33333333333334
Episode 299: -110.33333333333333


In [19]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')